In [ ]:
!pip install datasets
import pandas as pd
from datasets import Dataset, load_dataset, load_from_disk, concatenate_datasets
import os
import threading
import requests
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from io import BytesIO
import shutil

from PIL import PngImagePlugin
LARGE_ENOUGH_NUMBER = 100
PngImagePlugin.MAX_TEXT_CHUNK = LARGE_ENOUGH_NUMBER * (1024**2)

# Import Data

In [ ]:
# os.makedirs("/kaggle/working/Google_Conceptual_Caption_Dataset_Images_Captions", exist_ok=True)

# source_path = "/kaggle/input/image-retrieval-clip-training-conceptual-caption"  # Replace with the actual source file path
# destination_path = "/kaggle/working/Google_Conceptual_Caption_Dataset_Images_Captions"  # Replace with the actual destination folder path

# # filenames = ["/Google_Conceptual_Caption_Dataset_Images_Captions/dataset.arrow", "/Google_Conceptual_Caption_Dataset_Images_Captions/dataset_info.json", "/Google_Conceptual_Caption_Dataset_Images_Captions/state.json"]

# import os; filenames = os.listdir(source_path+"/Google_Conceptual_Caption_Dataset_Images_Captions")

# filenames = [os.path.join(source_path, "Google_Conceptual_Caption_Dataset_Images_Captions/"+file) for file in filenames]

# for file in filenames:
#     # Copy each file from source to destination
#     shutil.copy(file, destination_path)
    
# dataset = load_from_disk('/kaggle/working/Google_Conceptual_Caption_Dataset_Images_Captions')

In [ ]:
num_processors = 1

In [ ]:
from datasets import Dataset, load_dataset

dataset = load_dataset("shirsh10mall/Image_Captioning_Dataset", streaming=False ) # split="train"
dataset

In [ ]:
# yo_ignore = Image.fromarray(np.zeros((8,8,3),dtype=np.uint8))
dataset = dataset.filter(lambda instance, index: instance["drop_this_row"] == "no" and (index<7000 or index>8000), with_indices=True, num_proc=num_processors)
dataset = dataset.remove_columns(["drop_this_row","image_url"])
dataset

# Display Data

In [ ]:
# for index in range(5,10):
#     # Display the image using matplotlib
#     image_array = dataset["train"][index]["image_data"]
#     caption = dataset["train"][index]["caption"]
#     plt.imshow(image_array)
#     plt.axis('off')  # Turn off axes
#     print("\n\n\n Caption: ", caption )
#     plt.show()

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import CLIPProcessor, CLIPModel, CLIPImageProcessor
import torchvision.transforms as transforms

os.environ["TOKENIZERS_PARALLELISM"] = "true"  # Set to "false" if needed

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained CLIP model and tokenizer
model_name = "openai/clip-vit-base-patch16" 
tokenizer = CLIPProcessor.from_pretrained(model_name)
processor = CLIPImageProcessor.from_pretrained(model_name)

max_length = 22

model = CLIPModel.from_pretrained("shirsh10mall/Fine_Tuned_CLIP_Model") # 
model.to(device)
model.eval()

In [ ]:
# # Define your dataset class
# class ImageCaptionDataset(Dataset):
#     def __init__(self, dataset, tokenizer, max_length=64):
#         self.dataset = dataset
#         self.tokenizer = tokenizer
#         self.max_length = max_length
#         self.transform = transforms.Compose([   # transforms.Resize(desired_size),  # Resize the PIL image
#                                                 transforms.ToTensor()  # Convert the PIL image back to tensor
#                                             ])
#     def __len__(self):
#         return len(self.dataset)

#     def __getitem__(self, idx):
#         instance = self.dataset[idx]
#         image = self.transform( instance["image_data"] )
#         if image.shape[0]==1:
#             image = torch.stack( [image]*3 , dim=1)
#         elif image.shape[0]==4:
#             image = image[:3, :, :]
            
#         image = processor(images=image, return_tensors="pt").pixel_values
        
#         caption = instance["caption"]
                
#         text_inputs = self.tokenizer(caption, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')
        
#         return { 'pixel_values': image.squeeze(), 'input_ids': text_inputs['input_ids'].squeeze(),
#                     'attention_mask': text_inputs['attention_mask'].squeeze() }
    

# train_dataset = ImageCaptionDataset( dataset, tokenizer, max_length )
# train_dataloader = DataLoader( train_dataset, batch_size=1, shuffle=False , prefetch_factor=2, num_workers=2 )

In [ ]:
transform = transforms.Compose([ transforms.Resize((224,224)), transforms.ToTensor() ])

def create_image_embeddings(instance):
    image = transform( instance["image_data"] )
    if image.shape[0]==1:
            image = torch.stack( [image]*3 , dim=1)
            image = image.squeeze()
            
    elif image.shape[0]==4:
        image = image[:3, :, :]

    # image = processor(images=image, return_tensors="pt").pixel_values
    text_inputs = tokenizer(instance["caption"], padding='max_length', truncation=True, max_length=max_length, return_tensors='pt')

    with torch.no_grad():
        outputs = model( pixel_values=image.unsqueeze(0).to(device) , input_ids = text_inputs['input_ids'].to(device),
                            attention_mask=text_inputs['attention_mask'].to(device) )
    

    instance["image_embeddings"] =  outputs.image_embeds.cpu().numpy()[0]
    instance["text_embeddings"] = outputs.text_embeds.cpu().numpy()[0]

    return instance

In [ ]:
dataset

In [ ]:
# dataset = dataset.add_column("image_embeddings", image_embeddings_list)

dataset = dataset.map( create_image_embeddings, num_proc=num_processors ) # num_processors

In [ ]:
# from tqdm import tqdm 

# image_embeddings_list = [0]*len(train_dataloader)

# model.to(device)
# batch_idx = 0
# for batch in tqdm(train_dataloader):
#     pixel_values = batch['pixel_values'].to(device)
#     input_ids = batch['input_ids'].to(device)
#     attention_mask = batch['attention_mask'].to(device)
#     with torch.no_grad():
#         outputs = model(pixel_values=pixel_values, input_ids=input_ids, attention_mask=attention_mask)
#         image_embeddings_list[batch_idx] = outputs.image_embeds.cpu().numpy()[0]
#     batch_idx=batch_idx+1

In [ ]:
# import sys

# def get_list_memory_usage(my_list):
#     size_bytes = sys.getsizeof(my_list)
#     size_mb = size_bytes / (1024 ** 2)
#     return size_bytes, size_mb

# bytes_used, mb_used = get_list_memory_usage(image_embeddings_list)
# print(f"List occupies {bytes_used} bytes or approximately {mb_used:.4f} MB")

In [ ]:
dataset.save_to_disk("/kaggle/working/Image_Captioning_GCC_Embeddings")
dataset

In [ ]:
from huggingface_hub import notebook_login
from huggingface_hub import login

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("SHIRSH_HUGGINGFACE_API_KEY")

login(token=secret_value_0, write_permission=True)

In [ ]:
try:
    dataset.push_to_hub("shirsh10mall/Image_Captioning_GCC_Embeddings")
except:
    print("skipped")

In [ ]:
# dataset.save_to_disk("/kaggle/working/Google_Conceptual_Caption_Dataset_Images_Embeddings_Captions")
# dataset

In [ ]:
# def calculate_similarity_scores(query_encoding,image_embeddings):
#     similarity_score = torch.cosine_similarity(query_encoding, torch.tensor(image_embeddings).to(device), dim=-1).item()
#     return similarity_score

In [ ]:
# # Function to perform image retrieval

# max_length = 32

# def image_retrieval(query, dataset, counter_n=3, display=False, search_type="normal"):
#     # Encode the query text
#     query_input = tokenizer(query, padding='max_length', truncation=True, max_length=max_length, return_tensors='pt')
#     query_input.to(device)
#     query_encoding = model.get_text_features(**query_input)
    
#     if search_type=="normal":
#         # Calculate similarity scores between query and images
#         dataset = dataset.map( lambda instance: { "similarity_scores": calculate_similarity_scores(query_encoding,instance["image_embeddings"]) } )

#         # Sort images by similarity scores
#         counter_n = 0
#         sorted_images = []
#         for index in torch.argsort(torch.tensor(dataset["similarity_scores"]), descending=True):
#             counter_n = counter_n + 1
#             sorted_images.append( dataset[int(index)]["image_data"] )
#             if counter_n>=3:
#                 break
#         dataset = dataset.remove_columns(["similarity_scores"])
#     # elif search_type=="faiss":
    
#     if display:
#         for i, image in enumerate(sorted_images):
#             plt.imshow(image)
#             plt.axis('off')  # Turn off axes
#             print("\n\n\n" )
#             plt.show()
    
#     return sorted_images, dataset  # Return top 3 similar images

In [ ]:
# # User input query
# query = "dogs"

# # Perform image retrieval
# retrieved_images, dataset = image_retrieval(query, dataset, display=True, counter_n=3)

In [ ]:
# # User input query
# query = "game"

# # Perform image retrieval
# retrieved_images, dataset = image_retrieval(query, dataset, display=True, counter_n=3)

In [ ]:
# # User input query
# query = "sport"

# # Perform image retrieval
# retrieved_images, dataset = image_retrieval(query, dataset, display=True, counter_n=3)

In [ ]:
# !pip install faiss-cpu
# !pip install faiss-gpu

# dataset.add_faiss_index(column="image_embeddings")
# dataset